# Tree with state propagation

In [1]:
%matplotlib widget
import bmcs_utils.api as bu
import traits.api as tr
from bmcs_utils.model import Model

In [2]:
scd = False

Each model can be regarded as a tree of model components.
Let us define models `TopModel`, `InterimModel` and `SubModel`.

In [3]:
class SubModel(Model):
    name = 'submodel'
    
    length = bu.Float(2, GEO=True)

    ipw_view = bu.View(
        bu.Item('length')
    )

In [4]:
sm = SubModel()
sm.state_change_debug = True

In [5]:
sm.length = 3

value_changed <__main__.SubModel object at 0x7fe455f88a90> TraitChangeEvent(object=<__main__.SubModel object at 0x7fe455f88a90>, name='length', old=2, new=3.0)


Let us define a more complex model

In [6]:
class InterimModel(Model):
    sm = bu.Instance(SubModel,())
    
    stiffness = tr.Property(bu.Float, depends_on='state_changed')
    @tr.cached_property
    def _get_stiffness(self):
        return self.sm.length * 10
    
    depends_on = ['sm']
    
    ipw_view = bu.View(
        bu.Item('stiffness', readonly=True)
    )

In [7]:
im = InterimModel(sm=sm)
im.state_change_debug = True

In [8]:
im.sm.length = 8

value_changed <__main__.SubModel object at 0x7fe455f88a90> TraitChangeEvent(object=<__main__.SubModel object at 0x7fe455f88a90>, name='length', old=3.0, new=8.0)
value_changed <__main__.InterimModel object at 0x7fe4568c7ba0> Notification from child <__main__.SubModel object at 0x7fe455f88a90>


In [9]:
class TopLevelModelEagerGraphChange(Model):
    name = 'tm'
    
    sm = bu.Instance(SubModel, ())
    def _sm_default(self):
        sm = SubModel()
        self.set_sm(sm)
        return sm
    def _sm_changed(self):
        self.set_sm(self.sm)
    def set_sm(self, sm):
        self.im.sm = sm
        self.im2.sm = sm
        
    im = bu.Instance(InterimModel, ())
    im2 = bu.Instance(InterimModel, ())

    time = bu.Float(400, ALG=True)

    tree = ['sm', 'im', 'im2']


In [10]:
tl = TopLevelModelEagerGraphChange()
tl.state_change_debug = True

In [11]:
tl.sm = SubModel()
tl.sm, tl.im2.sm, tl.im.sm

parent <__main__.InterimModel object at 0x7fe456151760> changing child from child <__main__.SubModel object at 0x7fe455e76340> to <__main__.SubModel object at 0x7fe455e9dbc0>
graph_changed <__main__.InterimModel object at 0x7fe456151760> Notification from child <__main__.SubModel object at 0x7fe455e9dbc0>
graph_changed <__main__.TopLevelModelEagerGraphChange object at 0x7fe455e75350> Notification from child <__main__.InterimModel object at 0x7fe456151760>
parent <__main__.InterimModel object at 0x7fe455e8a750> changing child from child <__main__.SubModel object at 0x7fe455e76340> to <__main__.SubModel object at 0x7fe455e9dbc0>
graph_changed <__main__.InterimModel object at 0x7fe455e8a750> Notification from child <__main__.SubModel object at 0x7fe455e9dbc0>
graph_changed <__main__.TopLevelModelEagerGraphChange object at 0x7fe455e75350> Notification from child <__main__.InterimModel object at 0x7fe455e8a750>
parent <__main__.TopLevelModelEagerGraphChange object at 0x7fe455e75350> changin

(<__main__.SubModel at 0x7fe455e9dbc0>,
 <__main__.SubModel at 0x7fe455e9dbc0>)

In [12]:
tl.im.parents, tl.im2.parents, tl.sm.parents

({<__main__.TopLevelModelEagerGraphChange at 0x7fe455e75350>},
 {<__main__.TopLevelModelEagerGraphChange at 0x7fe455e75350>},
 {<__main__.InterimModel at 0x7fe455e8a750>,
  <__main__.TopLevelModelEagerGraphChange at 0x7fe455e75350>})

In [13]:
tl.im.sm.length = 10

value_changed <__main__.SubModel object at 0x7fe455e9dbc0> TraitChangeEvent(object=<__main__.SubModel object at 0x7fe455e9dbc0>, name='length', old=2, new=10.0)
value_changed <__main__.TopLevelModelEagerGraphChange object at 0x7fe455e75350> Notification from child <__main__.SubModel object at 0x7fe455e9dbc0>
value_changed <__main__.InterimModel object at 0x7fe455e8a750> Notification from child <__main__.SubModel object at 0x7fe455e9dbc0>
value_changed <__main__.TopLevelModelEagerGraphChange object at 0x7fe455e75350> Notification from child <__main__.InterimModel object at 0x7fe455e8a750>
value_changed <__main__.InterimModel object at 0x7fe456151760> Notification from child <__main__.SubModel object at 0x7fe455e9dbc0>
value_changed <__main__.TopLevelModelEagerGraphChange object at 0x7fe455e75350> Notification from child <__main__.InterimModel object at 0x7fe456151760>


In [14]:
tl.im, tl.im2, tl.im.sm.parents

(<__main__.InterimModel at 0x7fe456151760>,
 {<__main__.InterimModel at 0x7fe455e8a750>,
  <__main__.TopLevelModelEagerGraphChange at 0x7fe455e75350>})

In [15]:
tl.interact()

In [18]:
im.sm.length

8.0

In [19]:
class TopLevelModelLazyGraphChange(Model):
    name = 'tm'
    
    sm = bu.Instance(SubModel, ())
        
    im = tr.Property(bu.Instance(InterimModel), depends_on='sm')
    @tr.cached_property
    def _get_im(self):
        print('im')
        return InterimModel(sm=self.sm)

    im2 = tr.Property(bu.Instance(InterimModel), depends_on='sm')
    @tr.cached_property
    def _get_im2(self):
        print('im2')
        return InterimModel(sm=self.sm)

    time = bu.Float(400, ALG=True)

    tree = ['sm', 'im', 'im2']

In [20]:
tl = TopLevelModelEagerGraphChange()
tl.sm.parents, tl.children_traits, tl.im.children_traits

({<__main__.InterimModel at 0x7fe44d8a9760>,
  <__main__.TopLevelModelEagerGraphChange at 0x7fe44ddf80e0>},
 {'sm': <__main__.SubModel at 0x7fe44d8a8180>,
  'im': <__main__.InterimModel at 0x7fe44d8a9760>,
  'im2': <__main__.InterimModel at 0x7fe44ddfad40>},
 {'sm': <__main__.SubModel at 0x7fe44d8aae30>})

In [21]:
tl.interact()

Add change the component and check if the changes of stiffness in InterimModel get updated

In [22]:
tl.sm = SubModel()
tl.sm.length = 30
tl.im.stiffness

300.0